<a href="https://colab.research.google.com/github/robertej19/icarus/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install feedparser

import feedparser
import geopy.distance

     |████████████████████████████████| 81 kB 3.8 MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=39445687c8b65ea0dfba7740403e36694527e5d91d9fcc2f24dac67479ac82d8
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built sgmllib3k


In [4]:
######## BASIC WORKING EXAMPLE 1 
NewsFeed = feedparser.parse("https://inciweb.nwcg.gov/feeds/rss/incidents/")


##Run this as an example to see what is going on:
first_entry = NewsFeed.entries[0]  #"NewsFeed" is a list of dictionaries, we want the first dictionary
print(first_entry.keys()) #Show the keys of the dictionary
##After running the above, we should see that the keys are:

##['title', 'title_detail', 'published', 'published_parsed', 
##'where', 'geo_lat', 'geo_long', 'links', 'link', 'id', 
##'guidislink', 'summary', 'summary_detail'])


dict_keys(['title', 'title_detail', 'published', 'published_parsed', 'where', 'geo_lat', 'geo_long', 'links', 'link', 'id', 'guidislink', 'summary', 'summary_detail'])


In [6]:
######## BASIC WORKING EXAMPLE 2
# #As example, SNRA lat long @ alturas trailhead and see what incidents are nearby
target_lat = 43.89798 
target_long = -114.90373
target_coords = (target_lat,target_long)

for entry_number, entry in enumerate(NewsFeed.entries):
    #print(entry_number)
    #print(entry['geo_lat'])
    #print(entry['published'])
    #print(entry['geo_long'])
    incident_coords = (entry['geo_lat'],entry['geo_long'])

    distance_km = geopy.distance.distance(target_coords, incident_coords).km
    distance_miles = distance_km*0.621371
    if distance_miles <= 70:
      print("Incident {} is {:.2f} miles away".format(entry['title'],distance_miles))



Incident Jakes Gulch (Wildfire) is 3.99 miles away
Incident Scarface Fire (Wildfire) is 54.41 miles away
Incident Boundary Fire (Wildfire) is 48.57 miles away
Incident Deer Fire (Wildfire) is 53.65 miles away


In [7]:
######## BASIC WORKING EXAMPLE 3
# Now lets wrap this up into a function

def GetFiresCloseTo(target_lat=44.5902,target_long=104.7146,incident_radius_miles=50):
    NewsFeed = feedparser.parse("https://inciweb.nwcg.gov/feeds/rss/incidents/")
    
    target_coords = (target_lat,target_long)

    entries_within_radius = []
    for entry_number, entry in enumerate(NewsFeed.entries):
        incident_coords = (entry['geo_lat'],entry['geo_long'])
        distance_km = geopy.distance.distance(target_coords, incident_coords).km
        distance_miles = distance_km*0.621371
        if distance_miles <= incident_radius_miles:
            entry['distance_miles'] = distance_miles
            entries_within_radius.append(entry)

    sorted_entries_within_radius = sorted(entries_within_radius, key = lambda i: i['distance_miles'])

    return sorted_entries_within_radius

In [8]:
# We will use SNRA again and get a list returned of incidents with the specified radius
target_lat_1 = 43.89798 
target_long_1 = -114.90373
incident_radius_miles_1 = 10
sorted_entries_within_radius = GetFiresCloseTo(target_lat=target_lat_1,
                                                target_long=target_long_1,
                                                incident_radius_miles=incident_radius_miles_1)

#Now, lets print those out. We can put this script on a server, and have it send emails instead of printing out results.
if not sorted_entries_within_radius: #i.e. if there are no entries within the defined radius:
    print("No incidents have been reported within a radius of {} miles of ({},{})".format(incident_radius_miles_1,
            target_lat_1,target_long_1))
else:
    for entry in sorted_entries_within_radius:
        print("\nIncident found {:.2f} miles from target location\n".format(entry['distance_miles']))
        print("Incident Name: {}\n".format(entry['title']))
        print("Date Published: {}\n".format(entry['published']))
        print("Latitude: {} Longitude : {}\n".format(entry['geo_lat'],entry['geo_long']))
        print("Incident Summary: {} \n \n".format(entry['summary']))



Incident found 3.99 miles from target location

Incident Name: Jakes Gulch (Wildfire)

Date Published: Sat, 11 Sep 2021 11:21:27 -05:00

Latitude: 43.8575 Longitude : -114.96083333333

Incident Summary: Low RH and increased winds contributed to the fire kicking up a little yesterday. There was some group tree torching as well as short range spotting that jumped over the containment lines. There was about a 40 acre increase and between the growth and the fact it jumped the containment line it brings containment back to 0%. The fire moved to the west toward the head water of the north fork of Ross Creek, away from any structures. A type 3 helicopter arrived on the fire yesterday bringing the total to 6 helicopters, 2 seats and 2 type 1 airtankers. Another type 3 incident command team was ordered but none were available. There are still a lot of fires burning in the west and it is hard to get teams, crews and equipment. However, a type 2 team was available, so they were ordered and will 